In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 19 17:40:24 2020

@author: thomasvermaelen2
"""
import pandas as pd
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split

""""os.chdir("./Documents/ESCP/Machine Learning in Python/Assignment 2")"""

train = pd.read_csv("train.csv")
test = pd.read_csv("test1.csv")

train = train.drop(columns=['visitTime','purchaseTime','C1','C3','C10'])
test = test.drop(columns=['visitTime','purchaseTime','C1','C3','C10'])

In [6]:
X_train = train.drop(["label"], axis = 1)
y_train = (train["label"] >0)*1
X_test = test.drop(["label"], axis =1)
y_test = (test["label"]>0)*1
y_test = y_test.values

In [7]:
nb_class = len(set(y_train))
id_toTrain =np.array([np.where(y_train ==i)[0] for i in range(nb_class)])
size_max = [len(id_toTrain[i]) for i in range(nb_class)]
print("before resampling")
print(size_max)

before resampling
[31371, 57]


In [8]:
blc = 150
for i in range(len(size_max)):
    if size_max[i] > blc:
        size_max[i] = int(blc*(np.log10(size_max[i]/blc)+1))
    else:
        size_max[i] = int(blc/(np.log10(blc/size_max[i])+1))

print("after resampling")
print(size_max)

after resampling
[498, 105]


In [9]:
for i in range(nb_class):
    if len(id_toTrain[i]) > size_max[i]:
        id_toTrain[i], tmp = train_test_split(id_toTrain[i], test_size=1-size_max[i]/len(id_toTrain[i]))
    else:
        id_toTrain[i] = np.concatenate((id_toTrain[i], id_toTrain[i][np.random.randint(len(id_toTrain[i]), size=int(size_max[i]-len(id_toTrain[i])))]))

id_toTrain = np.concatenate(id_toTrain)
X_toTrain = X_train.values[id_toTrain]
y_toTrain = y_train.values[id_toTrain]

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, accuracy_score

clf = RandomForestClassifier(n_estimators=1000, random_state=42)
clf.fit(X_toTrain, y_toTrain)
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       1.00      0.99      1.00     32086
           1       0.00      0.00      0.00         0

    accuracy                           0.99     32086
   macro avg       0.50      0.50      0.50     32086
weighted avg       1.00      0.99      1.00     32086



/Users/thomasvermaelen/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
import numpy as np
pred = clf.predict_proba(X_test)
ids = np.array(X_test["id"])
pred = np.array(pred[:,1])

results = pd.DataFrame({'id': ids, 'prob': pred}, columns=['id', 'prob'])
results

,id,prob
0,1,0.004
1,2,0.004
2,3,0.006
3,4,0.010
4,5,0.009
...,...,...
32081,32082,0.118
32082,32083,0.046
32083,32084,0.046
32084,32085,0.236


In [32]:
results.to_csv("prediction.csv", index =False)